> git config pull.rebase true
>- Git will automatically rebase your local commits on top of the remote branch's commits when you perform a 'git pull' operation.
>- Rebasing helps to maintain a linear commit history, which can make the project's history easier to understand and navigate.

> git config pull.ff only
>- A fast-forward merge occurs when the remote branch's commit history is a direct ancestor of the local branch's commit history. 
>- In this case, Git can simply "fast-forward" the local branch to the remote branch's commit without creating a merge commit. This helps to keep the commit history clean and linear.

> git pull
>- This command fetches the latest commits from the remote repository and merges them into the local repository.
>- Since the "pull.rebase" and "pull.ff" options were set, Git will automatically rebase the local commits on top of the remote commits and perform a fast-forward merge if possible.
 

In [ ]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

In [ ]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

In [ ]:
ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
ALL_TICKERS

['MSFT',
 'AAPL',
 'GOOG',
 'NVDA',
 'AMZN',
 'META',
 'BRK-B',
 'LLY',
 'AVGO',
 'V',
 'JPM',
 'NVO',
 'MC.PA',
 'ASML',
 'RMS.PA',
 'OR.PA',
 'SAP',
 'ACN',
 'TTE',
 'SIE.DE',
 'IDEXY',
 'CDI.PA',
 'RELIANCE.NS',
 'TCS.NS',
 'HDB',
 'BHARTIARTL.NS',
 'IBN',
 'SBIN.NS',
 'LICI.NS',
 'INFY',
 'ITC.NS',
 'HINDUNILVR.NS',
 'LT.NS']

In [ ]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed


2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed


32 LT.NS


[*********************100%%**********************]  1 of 1 completed


In [ ]:
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'


In [ ]:
stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS))

In [ ]:
# count of observations between US-EU-INDIA stocks
stocks_df.ticker_type.value_counts()

ticker_type
US       80639
EU       76986
INDIA    63265
Name: count, dtype: int64

In [ ]:
# unique tickers
stocks_df.Ticker.nunique()

33

In [ ]:
# count of observations by stock
stocks_df.Ticker.value_counts()

Ticker
LLY              13087
JPM              11121
AAPL             10933
NVO              10838
MSFT              9607
CDI.PA            8321
TTE               8185
ASML              7330
SAP               7201
HINDUNILVR.NS     7115
SBIN.NS           7113
RELIANCE.NS       7112
ITC.NS            7112
SIE.DE            7039
BRK-B             7038
AMZN              6780
NVDA              6356
INFY              6323
MC.PA             6251
RMS.PA            6251
OR.PA             6251
IBN               6058
ACN               5728
HDB               5727
LT.NS             5420
BHARTIARTL.NS     5417
TCS.NS            5388
GOOG              4955
V                 4054
AVGO              3705
IDEXY             3591
META              3003
LICI.NS            480
Name: count, dtype: int64

In [ ]:
stocks_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
AAPL,US,1980-12-12,2024-04-25
ACN,EU,2001-07-19,2024-04-25
AMZN,US,1997-05-15,2024-04-24
ASML,EU,1995-03-15,2024-04-25
AVGO,US,2009-08-06,2024-04-25
BHARTIARTL.NS,INDIA,2002-07-01,2024-04-25
BRK-B,US,1996-05-09,2024-04-25
CDI.PA,EU,1992-01-27,2024-04-25
GOOG,US,2004-08-19,2024-04-25


In [ ]:
#  average growth 365days
stocks_df[stocks_df.Year>=2020].groupby(by=['Year','ticker_type']).growth_365d.mean()

Year  ticker_type
2020  EU             1.245388
      INDIA          1.101896
      US             1.484070
2021  EU             1.582170
      INDIA          1.546424
      US             1.797571
2022  EU             1.191169
      INDIA          1.379889
      US             1.182379
2023  EU             1.246533
      INDIA          1.201110
      US             1.203253
2024  EU             1.650721
      INDIA          1.305762
      US             2.153187
Name: growth_365d, dtype: float64

In [ ]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

In [ ]:
# filtering only on stats after 2000
stocks_df[stocks_df.Date>='2000-01-01'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182423 entries, 3490 to 220889
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Open                          182423 non-null  float64       
 1   High                          182423 non-null  float64       
 2   Low                           182423 non-null  float64       
 3   Close                         182423 non-null  float64       
 4   Adj Close                     182423 non-null  float64       
 5   Volume                        182423 non-null  int64         
 6   Ticker                        182423 non-null  object        
 7   Year                          182423 non-null  int32         
 8   Month                         182423 non-null  int32         
 9   Weekday                       182423 non-null  int32         
 10  Date                          182423 non-null  datetime64[ns]
 11  growth_1d      

In [ ]:
stocks_df[stocks_df.Date=='2024-04-01'].tail()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_90d,growth_365d,growth_future_5d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_5d_future,ticker_type
194903,925.000000,984.450012,924.000000,970.599976,970.599976,4863536,LICI.NS,2024,4,0,...,1.599113,1.556710,1.026891,906.085004,953.950003,0,0.062281,1034.237352,1,INDIA
201224,17.889999,17.920000,17.500000,17.580000,17.580000,14601600,INFY,2024,4,0,...,1.003998,1.024508,1.015927,18.156000,18.777500,0,0.023891,13.940634,1,INDIA
208338,429.100006,431.350006,425.750000,426.700012,426.700012,9382308,ITC.NS,2024,4,0,...,0.987044,1.314436,1.005625,422.235004,416.537502,1,0.013124,130.246651,1,INDIA
215453,2270.000000,2295.000000,2262.000000,2285.899902,2285.899902,1427873,HINDUNILVR.NS,2024,4,0,...,0.917590,0.894051,0.992585,2266.539990,2327.240015,0,0.014436,1049.970071,0,INDIA
220873,3780.850098,3851.850098,3780.850098,3838.000000,3838.000000,1905190,LT.NS,2024,4,0,...,1.257886,2.010850,0.992144,3626.594971,3618.109985,1,0.018499,2188.619806,0,INDIA


In [ ]:
ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
ALL_TICKERS

['MSFT',
 'AAPL',
 'GOOG',
 'NVDA',
 'AMZN',
 'META',
 'BRK-B',
 'LLY',
 'AVGO',
 'V',
 'JPM',
 'NVO',
 'MC.PA',
 'ASML',
 'RMS.PA',
 'OR.PA',
 'SAP',
 'ACN',
 'TTE',
 'SIE.DE',
 'IDEXY',
 'CDI.PA',
 'RELIANCE.NS',
 'TCS.NS',
 'HDB',
 'BHARTIARTL.NS',
 'IBN',
 'SBIN.NS',
 'LICI.NS',
 'INFY',
 'ITC.NS',
 'HINDUNILVR.NS',
 'LT.NS']

In [ ]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed


2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed


32 LT.NS


[*********************100%%**********************]  1 of 1 completed


In [ ]:
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'


In [ ]:
stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS))

In [ ]:
# count of observations between US-EU-INDIA stocks
stocks_df.ticker_type.value_counts()

ticker_type
US       80639
EU       76986
INDIA    63265
Name: count, dtype: int64

In [ ]:
# unique tickers
stocks_df.Ticker.nunique()

33

In [ ]:
# count of observations by stock
stocks_df.Ticker.value_counts()

Ticker
LLY              13087
JPM              11121
AAPL             10933
NVO              10838
MSFT              9607
CDI.PA            8321
TTE               8185
ASML              7330
SAP               7201
HINDUNILVR.NS     7115
SBIN.NS           7113
RELIANCE.NS       7112
ITC.NS            7112
SIE.DE            7039
BRK-B             7038
AMZN              6780
NVDA              6356
INFY              6323
MC.PA             6251
RMS.PA            6251
OR.PA             6251
IBN               6058
ACN               5728
HDB               5727
LT.NS             5420
BHARTIARTL.NS     5417
TCS.NS            5388
GOOG              4955
V                 4054
AVGO              3705
IDEXY             3591
META              3003
LICI.NS            480
Name: count, dtype: int64

In [ ]:
stocks_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
AAPL,US,1980-12-12,2024-04-25
ACN,EU,2001-07-19,2024-04-25
AMZN,US,1997-05-15,2024-04-24
ASML,EU,1995-03-15,2024-04-25
AVGO,US,2009-08-06,2024-04-25
BHARTIARTL.NS,INDIA,2002-07-01,2024-04-25
BRK-B,US,1996-05-09,2024-04-25
CDI.PA,EU,1992-01-27,2024-04-25
GOOG,US,2004-08-19,2024-04-25


In [ ]:
#  average growth 365days
stocks_df[stocks_df.Year>=2020].groupby(by=['Year','ticker_type']).growth_365d.mean()

Year  ticker_type
2020  EU             1.245388
      INDIA          1.101896
      US             1.484070
2021  EU             1.582170
      INDIA          1.546424
      US             1.797571
2022  EU             1.191169
      INDIA          1.379889
      US             1.182379
2023  EU             1.246533
      INDIA          1.201110
      US             1.203253
2024  EU             1.650721
      INDIA          1.305762
      US             2.153187
Name: growth_365d, dtype: float64

In [ ]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

In [ ]:
# filtering only on stats after 2000
stocks_df[stocks_df.Date>='2000-01-01'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182423 entries, 3490 to 220889
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Open                          182423 non-null  float64       
 1   High                          182423 non-null  float64       
 2   Low                           182423 non-null  float64       
 3   Close                         182423 non-null  float64       
 4   Adj Close                     182423 non-null  float64       
 5   Volume                        182423 non-null  int64         
 6   Ticker                        182423 non-null  object        
 7   Year                          182423 non-null  int32         
 8   Month                         182423 non-null  int32         
 9   Weekday                       182423 non-null  int32         
 10  Date                          182423 non-null  datetime64[ns]
 11  growth_1d      

In [ ]:
stocks_df[stocks_df.Date=='2024-04-01'].tail()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_90d,growth_365d,growth_future_5d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_5d_future,ticker_type
194903,925.000000,984.450012,924.000000,970.599976,970.599976,4863536,LICI.NS,2024,4,0,...,1.599113,1.556710,1.026891,906.085004,953.950003,0,0.062281,1034.237352,1,INDIA
201224,17.889999,17.920000,17.500000,17.580000,17.580000,14601600,INFY,2024,4,0,...,1.003998,1.024508,1.015927,18.156000,18.777500,0,0.023891,13.940634,1,INDIA
208338,429.100006,431.350006,425.750000,426.700012,426.700012,9382308,ITC.NS,2024,4,0,...,0.987044,1.314436,1.005625,422.235004,416.537502,1,0.013124,130.246651,1,INDIA
215453,2270.000000,2295.000000,2262.000000,2285.899902,2285.899902,1427873,HINDUNILVR.NS,2024,4,0,...,0.917590,0.894051,0.992585,2266.539990,2327.240015,0,0.014436,1049.970071,0,INDIA
220873,3780.850098,3851.850098,3780.850098,3838.000000,3838.000000,1905190,LT.NS,2024,4,0,...,1.257886,2.010850,0.992144,3626.594971,3618.109985,1,0.018499,2188.619806,0,INDIA


In [ ]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

In [ ]:
ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
ALL_TICKERS

['MSFT',
 'AAPL',
 'GOOG',
 'NVDA',
 'AMZN',
 'META',
 'BRK-B',
 'LLY',
 'AVGO',
 'V',
 'JPM',
 'NVO',
 'MC.PA',
 'ASML',
 'RMS.PA',
 'OR.PA',
 'SAP',
 'ACN',
 'TTE',
 'SIE.DE',
 'IDEXY',
 'CDI.PA',
 'RELIANCE.NS',
 'TCS.NS',
 'HDB',
 'BHARTIARTL.NS',
 'IBN',
 'SBIN.NS',
 'LICI.NS',
 'INFY',
 'ITC.NS',
 'HINDUNILVR.NS',
 'LT.NS']

In [ ]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed


2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed


32 LT.NS


[*********************100%%**********************]  1 of 1 completed


In [ ]:
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'


In [ ]:
stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS))

In [ ]:
# count of observations between US-EU-INDIA stocks
stocks_df.ticker_type.value_counts()

ticker_type
US       80639
EU       76986
INDIA    63265
Name: count, dtype: int64

In [ ]:
# unique tickers
stocks_df.Ticker.nunique()

33

In [ ]:
# count of observations by stock
stocks_df.Ticker.value_counts()

Ticker
LLY              13087
JPM              11121
AAPL             10933
NVO              10838
MSFT              9607
CDI.PA            8321
TTE               8185
ASML              7330
SAP               7201
HINDUNILVR.NS     7115
SBIN.NS           7113
RELIANCE.NS       7112
ITC.NS            7112
SIE.DE            7039
BRK-B             7038
AMZN              6780
NVDA              6356
INFY              6323
MC.PA             6251
RMS.PA            6251
OR.PA             6251
IBN               6058
ACN               5728
HDB               5727
LT.NS             5420
BHARTIARTL.NS     5417
TCS.NS            5388
GOOG              4955
V                 4054
AVGO              3705
IDEXY             3591
META              3003
LICI.NS            480
Name: count, dtype: int64

In [ ]:
stocks_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
AAPL,US,1980-12-12,2024-04-25
ACN,EU,2001-07-19,2024-04-25
AMZN,US,1997-05-15,2024-04-24
ASML,EU,1995-03-15,2024-04-25
AVGO,US,2009-08-06,2024-04-25
BHARTIARTL.NS,INDIA,2002-07-01,2024-04-25
BRK-B,US,1996-05-09,2024-04-25
CDI.PA,EU,1992-01-27,2024-04-25
GOOG,US,2004-08-19,2024-04-25


In [ ]:
#  average growth 365days
stocks_df[stocks_df.Year>=2020].groupby(by=['Year','ticker_type']).growth_365d.mean()

Year  ticker_type
2020  EU             1.245388
      INDIA          1.101896
      US             1.484070
2021  EU             1.582170
      INDIA          1.546424
      US             1.797571
2022  EU             1.191169
      INDIA          1.379889
      US             1.182379
2023  EU             1.246533
      INDIA          1.201110
      US             1.203253
2024  EU             1.650721
      INDIA          1.305762
      US             2.153187
Name: growth_365d, dtype: float64

In [ ]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

In [ ]:
# filtering only on stats after 2000
stocks_df[stocks_df.Date>='2000-01-01'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182423 entries, 3490 to 220889
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Open                          182423 non-null  float64       
 1   High                          182423 non-null  float64       
 2   Low                           182423 non-null  float64       
 3   Close                         182423 non-null  float64       
 4   Adj Close                     182423 non-null  float64       
 5   Volume                        182423 non-null  int64         
 6   Ticker                        182423 non-null  object        
 7   Year                          182423 non-null  int32         
 8   Month                         182423 non-null  int32         
 9   Weekday                       182423 non-null  int32         
 10  Date                          182423 non-null  datetime64[ns]
 11  growth_1d      

In [ ]:
stocks_df[stocks_df.Date=='2024-04-01'].tail()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_90d,growth_365d,growth_future_5d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_5d_future,ticker_type
194903,925.000000,984.450012,924.000000,970.599976,970.599976,4863536,LICI.NS,2024,4,0,...,1.599113,1.556710,1.026891,906.085004,953.950003,0,0.062281,1034.237352,1,INDIA
201224,17.889999,17.920000,17.500000,17.580000,17.580000,14601600,INFY,2024,4,0,...,1.003998,1.024508,1.015927,18.156000,18.777500,0,0.023891,13.940634,1,INDIA
208338,429.100006,431.350006,425.750000,426.700012,426.700012,9382308,ITC.NS,2024,4,0,...,0.987044,1.314436,1.005625,422.235004,416.537502,1,0.013124,130.246651,1,INDIA
215453,2270.000000,2295.000000,2262.000000,2285.899902,2285.899902,1427873,HINDUNILVR.NS,2024,4,0,...,0.917590,0.894051,0.992585,2266.539990,2327.240015,0,0.014436,1049.970071,0,INDIA
220873,3780.850098,3851.850098,3780.850098,3838.000000,3838.000000,1905190,LT.NS,2024,4,0,...,1.257886,2.010850,0.992144,3626.594971,3618.109985,1,0.018499,2188.619806,0,INDIA


In [ ]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

In [ ]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

In [ ]:
ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
ALL_TICKERS

['MSFT',
 'AAPL',
 'GOOG',
 'NVDA',
 'AMZN',
 'META',
 'BRK-B',
 'LLY',
 'AVGO',
 'V',
 'JPM',
 'NVO',
 'MC.PA',
 'ASML',
 'RMS.PA',
 'OR.PA',
 'SAP',
 'ACN',
 'TTE',
 'SIE.DE',
 'IDEXY',
 'CDI.PA',
 'RELIANCE.NS',
 'TCS.NS',
 'HDB',
 'BHARTIARTL.NS',
 'IBN',
 'SBIN.NS',
 'LICI.NS',
 'INFY',
 'ITC.NS',
 'HINDUNILVR.NS',
 'LT.NS']

In [ ]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed


2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed


32 LT.NS


[*********************100%%**********************]  1 of 1 completed


In [ ]:
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'


In [ ]:
stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS))

In [ ]:
# count of observations between US-EU-INDIA stocks
stocks_df.ticker_type.value_counts()

ticker_type
US       80639
EU       76986
INDIA    63265
Name: count, dtype: int64

In [ ]:
# unique tickers
stocks_df.Ticker.nunique()

33

In [ ]:
# count of observations by stock
stocks_df.Ticker.value_counts()

Ticker
LLY              13087
JPM              11121
AAPL             10933
NVO              10838
MSFT              9607
CDI.PA            8321
TTE               8185
ASML              7330
SAP               7201
HINDUNILVR.NS     7115
SBIN.NS           7113
RELIANCE.NS       7112
ITC.NS            7112
SIE.DE            7039
BRK-B             7038
AMZN              6780
NVDA              6356
INFY              6323
MC.PA             6251
RMS.PA            6251
OR.PA             6251
IBN               6058
ACN               5728
HDB               5727
LT.NS             5420
BHARTIARTL.NS     5417
TCS.NS            5388
GOOG              4955
V                 4054
AVGO              3705
IDEXY             3591
META              3003
LICI.NS            480
Name: count, dtype: int64

In [ ]:
stocks_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
AAPL,US,1980-12-12,2024-04-25
ACN,EU,2001-07-19,2024-04-25
AMZN,US,1997-05-15,2024-04-24
ASML,EU,1995-03-15,2024-04-25
AVGO,US,2009-08-06,2024-04-25
BHARTIARTL.NS,INDIA,2002-07-01,2024-04-25
BRK-B,US,1996-05-09,2024-04-25
CDI.PA,EU,1992-01-27,2024-04-25
GOOG,US,2004-08-19,2024-04-25


In [ ]:
#  average growth 365days
stocks_df[stocks_df.Year>=2020].groupby(by=['Year','ticker_type']).growth_365d.mean()

Year  ticker_type
2020  EU             1.245388
      INDIA          1.101896
      US             1.484070
2021  EU             1.582170
      INDIA          1.546424
      US             1.797571
2022  EU             1.191169
      INDIA          1.379889
      US             1.182379
2023  EU             1.246533
      INDIA          1.201110
      US             1.203253
2024  EU             1.650721
      INDIA          1.305762
      US             2.153187
Name: growth_365d, dtype: float64

In [ ]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

In [ ]:
# filtering only on stats after 2000
stocks_df[stocks_df.Date>='2000-01-01'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182423 entries, 3490 to 220889
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Open                          182423 non-null  float64       
 1   High                          182423 non-null  float64       
 2   Low                           182423 non-null  float64       
 3   Close                         182423 non-null  float64       
 4   Adj Close                     182423 non-null  float64       
 5   Volume                        182423 non-null  int64         
 6   Ticker                        182423 non-null  object        
 7   Year                          182423 non-null  int32         
 8   Month                         182423 non-null  int32         
 9   Weekday                       182423 non-null  int32         
 10  Date                          182423 non-null  datetime64[ns]
 11  growth_1d      

In [ ]:
stocks_df[stocks_df.Date=='2024-04-01'].tail()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_90d,growth_365d,growth_future_5d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_5d_future,ticker_type
194903,925.000000,984.450012,924.000000,970.599976,970.599976,4863536,LICI.NS,2024,4,0,...,1.599113,1.556710,1.026891,906.085004,953.950003,0,0.062281,1034.237352,1,INDIA
201224,17.889999,17.920000,17.500000,17.580000,17.580000,14601600,INFY,2024,4,0,...,1.003998,1.024508,1.015927,18.156000,18.777500,0,0.023891,13.940634,1,INDIA
208338,429.100006,431.350006,425.750000,426.700012,426.700012,9382308,ITC.NS,2024,4,0,...,0.987044,1.314436,1.005625,422.235004,416.537502,1,0.013124,130.246651,1,INDIA
215453,2270.000000,2295.000000,2262.000000,2285.899902,2285.899902,1427873,HINDUNILVR.NS,2024,4,0,...,0.917590,0.894051,0.992585,2266.539990,2327.240015,0,0.014436,1049.970071,0,INDIA
220873,3780.850098,3851.850098,3780.850098,3838.000000,3838.000000,1905190,LT.NS,2024,4,0,...,1.257886,2.010850,0.992144,3626.594971,3618.109985,1,0.018499,2188.619806,0,INDIA


In [ ]:
ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
ALL_TICKERS

['MSFT',
 'AAPL',
 'GOOG',
 'NVDA',
 'AMZN',
 'META',
 'BRK-B',
 'LLY',
 'AVGO',
 'V',
 'JPM',
 'NVO',
 'MC.PA',
 'ASML',
 'RMS.PA',
 'OR.PA',
 'SAP',
 'ACN',
 'TTE',
 'SIE.DE',
 'IDEXY',
 'CDI.PA',
 'RELIANCE.NS',
 'TCS.NS',
 'HDB',
 'BHARTIARTL.NS',
 'IBN',
 'SBIN.NS',
 'LICI.NS',
 'INFY',
 'ITC.NS',
 'HINDUNILVR.NS',
 'LT.NS']

In [ ]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed


2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed


32 LT.NS


[*********************100%%**********************]  1 of 1 completed


In [ ]:
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'


In [ ]:
stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS))

In [ ]:
# count of observations between US-EU-INDIA stocks
stocks_df.ticker_type.value_counts()

ticker_type
US       80639
EU       76986
INDIA    63265
Name: count, dtype: int64

In [ ]:
# unique tickers
stocks_df.Ticker.nunique()

33

In [ ]:
# count of observations by stock
stocks_df.Ticker.value_counts()

Ticker
LLY              13087
JPM              11121
AAPL             10933
NVO              10838
MSFT              9607
CDI.PA            8321
TTE               8185
ASML              7330
SAP               7201
HINDUNILVR.NS     7115
SBIN.NS           7113
RELIANCE.NS       7112
ITC.NS            7112
SIE.DE            7039
BRK-B             7038
AMZN              6780
NVDA              6356
INFY              6323
MC.PA             6251
RMS.PA            6251
OR.PA             6251
IBN               6058
ACN               5728
HDB               5727
LT.NS             5420
BHARTIARTL.NS     5417
TCS.NS            5388
GOOG              4955
V                 4054
AVGO              3705
IDEXY             3591
META              3003
LICI.NS            480
Name: count, dtype: int64

In [ ]:
stocks_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
AAPL,US,1980-12-12,2024-04-25
ACN,EU,2001-07-19,2024-04-25
AMZN,US,1997-05-15,2024-04-24
ASML,EU,1995-03-15,2024-04-25
AVGO,US,2009-08-06,2024-04-25
BHARTIARTL.NS,INDIA,2002-07-01,2024-04-25
BRK-B,US,1996-05-09,2024-04-25
CDI.PA,EU,1992-01-27,2024-04-25
GOOG,US,2004-08-19,2024-04-25


In [ ]:
#  average growth 365days
stocks_df[stocks_df.Year>=2020].groupby(by=['Year','ticker_type']).growth_365d.mean()

Year  ticker_type
2020  EU             1.245388
      INDIA          1.101896
      US             1.484070
2021  EU             1.582170
      INDIA          1.546424
      US             1.797571
2022  EU             1.191169
      INDIA          1.379889
      US             1.182379
2023  EU             1.246533
      INDIA          1.201110
      US             1.203253
2024  EU             1.650721
      INDIA          1.305762
      US             2.153187
Name: growth_365d, dtype: float64

In [ ]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

In [ ]:
# filtering only on stats after 2000
stocks_df[stocks_df.Date>='2000-01-01'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182423 entries, 3490 to 220889
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Open                          182423 non-null  float64       
 1   High                          182423 non-null  float64       
 2   Low                           182423 non-null  float64       
 3   Close                         182423 non-null  float64       
 4   Adj Close                     182423 non-null  float64       
 5   Volume                        182423 non-null  int64         
 6   Ticker                        182423 non-null  object        
 7   Year                          182423 non-null  int32         
 8   Month                         182423 non-null  int32         
 9   Weekday                       182423 non-null  int32         
 10  Date                          182423 non-null  datetime64[ns]
 11  growth_1d      

In [ ]:
stocks_df[stocks_df.Date=='2024-04-01'].tail()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_90d,growth_365d,growth_future_5d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_5d_future,ticker_type
194903,925.000000,984.450012,924.000000,970.599976,970.599976,4863536,LICI.NS,2024,4,0,...,1.599113,1.556710,1.026891,906.085004,953.950003,0,0.062281,1034.237352,1,INDIA
201224,17.889999,17.920000,17.500000,17.580000,17.580000,14601600,INFY,2024,4,0,...,1.003998,1.024508,1.015927,18.156000,18.777500,0,0.023891,13.940634,1,INDIA
208338,429.100006,431.350006,425.750000,426.700012,426.700012,9382308,ITC.NS,2024,4,0,...,0.987044,1.314436,1.005625,422.235004,416.537502,1,0.013124,130.246651,1,INDIA
215453,2270.000000,2295.000000,2262.000000,2285.899902,2285.899902,1427873,HINDUNILVR.NS,2024,4,0,...,0.917590,0.894051,0.992585,2266.539990,2327.240015,0,0.014436,1049.970071,0,INDIA
220873,3780.850098,3851.850098,3780.850098,3838.000000,3838.000000,1905190,LT.NS,2024,4,0,...,1.257886,2.010850,0.992144,3626.594971,3618.109985,1,0.018499,2188.619806,0,INDIA


In [ ]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

In [ ]:
ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
ALL_TICKERS

['MSFT',
 'AAPL',
 'GOOG',
 'NVDA',
 'AMZN',
 'META',
 'BRK-B',
 'LLY',
 'AVGO',
 'V',
 'JPM',
 'NVO',
 'MC.PA',
 'ASML',
 'RMS.PA',
 'OR.PA',
 'SAP',
 'ACN',
 'TTE',
 'SIE.DE',
 'IDEXY',
 'CDI.PA',
 'RELIANCE.NS',
 'TCS.NS',
 'HDB',
 'BHARTIARTL.NS',
 'IBN',
 'SBIN.NS',
 'LICI.NS',
 'INFY',
 'ITC.NS',
 'HINDUNILVR.NS',
 'LT.NS']

In [ ]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed


2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed


32 LT.NS


[*********************100%%**********************]  1 of 1 completed


In [ ]:
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'


In [ ]:
stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS))

In [ ]:
# count of observations between US-EU-INDIA stocks
stocks_df.ticker_type.value_counts()

ticker_type
US       80639
EU       76986
INDIA    63265
Name: count, dtype: int64

In [ ]:
# unique tickers
stocks_df.Ticker.nunique()

33

In [ ]:
# count of observations by stock
stocks_df.Ticker.value_counts()

Ticker
LLY              13087
JPM              11121
AAPL             10933
NVO              10838
MSFT              9607
CDI.PA            8321
TTE               8185
ASML              7330
SAP               7201
HINDUNILVR.NS     7115
SBIN.NS           7113
RELIANCE.NS       7112
ITC.NS            7112
SIE.DE            7039
BRK-B             7038
AMZN              6780
NVDA              6356
INFY              6323
MC.PA             6251
RMS.PA            6251
OR.PA             6251
IBN               6058
ACN               5728
HDB               5727
LT.NS             5420
BHARTIARTL.NS     5417
TCS.NS            5388
GOOG              4955
V                 4054
AVGO              3705
IDEXY             3591
META              3003
LICI.NS            480
Name: count, dtype: int64

In [ ]:
stocks_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
AAPL,US,1980-12-12,2024-04-25
ACN,EU,2001-07-19,2024-04-25
AMZN,US,1997-05-15,2024-04-24
ASML,EU,1995-03-15,2024-04-25
AVGO,US,2009-08-06,2024-04-25
BHARTIARTL.NS,INDIA,2002-07-01,2024-04-25
BRK-B,US,1996-05-09,2024-04-25
CDI.PA,EU,1992-01-27,2024-04-25
GOOG,US,2004-08-19,2024-04-25


In [ ]:
#  average growth 365days
stocks_df[stocks_df.Year>=2020].groupby(by=['Year','ticker_type']).growth_365d.mean()

Year  ticker_type
2020  EU             1.245388
      INDIA          1.101896
      US             1.484070
2021  EU             1.582170
      INDIA          1.546424
      US             1.797571
2022  EU             1.191169
      INDIA          1.379889
      US             1.182379
2023  EU             1.246533
      INDIA          1.201110
      US             1.203253
2024  EU             1.650721
      INDIA          1.305762
      US             2.153187
Name: growth_365d, dtype: float64

In [ ]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

In [ ]:
# filtering only on stats after 2000
stocks_df[stocks_df.Date>='2000-01-01'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182423 entries, 3490 to 220889
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Open                          182423 non-null  float64       
 1   High                          182423 non-null  float64       
 2   Low                           182423 non-null  float64       
 3   Close                         182423 non-null  float64       
 4   Adj Close                     182423 non-null  float64       
 5   Volume                        182423 non-null  int64         
 6   Ticker                        182423 non-null  object        
 7   Year                          182423 non-null  int32         
 8   Month                         182423 non-null  int32         
 9   Weekday                       182423 non-null  int32         
 10  Date                          182423 non-null  datetime64[ns]
 11  growth_1d      

In [ ]:
stocks_df[stocks_df.Date=='2024-04-01'].tail()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_90d,growth_365d,growth_future_5d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_5d_future,ticker_type
194903,925.000000,984.450012,924.000000,970.599976,970.599976,4863536,LICI.NS,2024,4,0,...,1.599113,1.556710,1.026891,906.085004,953.950003,0,0.062281,1034.237352,1,INDIA
201224,17.889999,17.920000,17.500000,17.580000,17.580000,14601600,INFY,2024,4,0,...,1.003998,1.024508,1.015927,18.156000,18.777500,0,0.023891,13.940634,1,INDIA
208338,429.100006,431.350006,425.750000,426.700012,426.700012,9382308,ITC.NS,2024,4,0,...,0.987044,1.314436,1.005625,422.235004,416.537502,1,0.013124,130.246651,1,INDIA
215453,2270.000000,2295.000000,2262.000000,2285.899902,2285.899902,1427873,HINDUNILVR.NS,2024,4,0,...,0.917590,0.894051,0.992585,2266.539990,2327.240015,0,0.014436,1049.970071,0,INDIA
220873,3780.850098,3851.850098,3780.850098,3838.000000,3838.000000,1905190,LT.NS,2024,4,0,...,1.257886,2.010850,0.992144,3626.594971,3618.109985,1,0.018499,2188.619806,0,INDIA


## Module 2 Homework

In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.

If not stated otherwise, please use the [Colab](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/02-dataframe-analysis/Module2_Colab_Working_with_the_data.ipynb) covered at the livestream to re-use the code snippets.

### Imports and Installs

In [1]:
!pip install yfinance

In [2]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

---
### Question 1: IPO Filings Web Scraping and Data Processing

**What's the total sum ($m) of 2023 filings that happened on Fridays?**

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/
Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs).
Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given).
You may be inspired by the function `extract_numbers()` in [Code Snippet 4], or you can write your own function to "parse" a string.
Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (`Date.dt.dayofweek()==4`). You should see 32 records in total, 25 of it is not null.

(additional: you can read about [S-1 IPO filing](https://www.dfinsolutions.com/knowledge-hub/thought-leadership/knowledge-resources/what-s-1-ipo-filing) to understand the context)


In [3]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [4]:
ipos_filings_df = ipo_dfs[0]

In [5]:
print("Number of features: {}".format(ipos_filings_df.shape[1]))
print("Number of samples: {}".format(ipos_filings_df.shape[0]))
print("Number of missing values: {}".format(ipos_filings_df.isnull().sum().sum()))
ipos_filings_df.isnull().sum()
ipos_filings_df.info()
ipos_filings_df.head(5)

Number of features: 5
Number of samples: 326
Number of missing values: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326 entries, 0 to 325
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     326 non-null    object
 1   Symbol          326 non-null    object
 2   Company Name    326 non-null    object
 3   Price Range     326 non-null    object
 4   Shares Offered  326 non-null    object
dtypes: object(5)
memory usage: 12.9+ KB


,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-


In [6]:
# Convert the 'Filing Date' to datetime() 
ipos_filings_df['Filing Date'] = pd.to_datetime(ipos_filings_df['Filing Date'],format='mixed')

# Convert the 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs).
ipos_filings_df['Shares Offered'] = ipos_filings_df['Shares Offered'].str.replace('-', 'nan').astype(float)


In [7]:
# Define a new field 'Avg_price' in the ipos_filings dataframe based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given). You may be inspired by the below function `extract_numbers()` or you can write your own function to "parse" a string.
import pandas as pd
import numpy as np
import re

def extract_numbers(price_range):
    numbers = re.findall(r'\d+\.?\d*', price_range)
    return [float(n) for n in numbers]

ipos_filings_df['Avg_price'] = ipos_filings_df['Price Range'].apply(lambda x: np.nan if pd.isna(x) else np.mean(extract_numbers(x)) if '-' in str(x) else extract_numbers(x)[0])


/Users/jschulle/miniconda3/envs/berlin/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [8]:
# Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)
ipos_filings_df['Shares_offered_value'] = ipos_filings_df['Shares Offered'] * ipos_filings_df['Avg_price']

### Question 1: Answer

In [9]:
# Find the number of filings during 2023, which happened on Fridays (`Date.dt.dayofweek()==4`).  
# You should see 32 records in total, 25 of it is not null.

# Filter for 2023 Fridays
friday_filings = ipos_filings_df[
    (ipos_filings_df['Filing Date'].dt.year == 2023) &  
    (ipos_filings_df['Filing Date'].dt.dayofweek == 4)
]

# Calculate total value  
total_value = friday_filings['Shares_offered_value'].sum()

# Format output
print(f"Total IPO value for 2023 Fridays: {int(total_value/1e6)} million")
print(f"Total IPO count for 2023 Fridays: {friday_filings['Filing Date'].count()}")
print(f"Total NULL IPO count for 2023 Fridays: {friday_filings['Shares_offered_value'].isnull().sum()}")


Total IPO value for 2023 Fridays: 285 million
Total IPO count for 2023 Fridays: 32
Total NULL IPO count for 2023 Fridays: 7


---
### Question 2:  IPOs "Fixed days hold" strategy


**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**


Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). 
Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance. 

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks.
Some of the tickers like 'DYCQ' and 'LEGT' were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day).
Find X, when the 75% quantile growth (among 185 investments) is the highest. 

HINTs:
* You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
* You can use the `DataFrame.describe()` function to get mean, min, max, 25-50-75% quantiles.


Additional: 
* You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
* What's your recommendation: Do you suggest pursuing this strategy for an optimal X?


In [10]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipos_2023 = pd.read_html(response.text)
df_ipos_2023 = ipos_2023[0]

url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipos_2024 = pd.read_html(response.text)
df_ipos_2024 = ipos_2024[0]


In [11]:
df_ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [12]:
df_ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      64 non-null     object
 1   Symbol        64 non-null     object
 2   Company Name  64 non-null     object
 3   IPO Price     64 non-null     object
 4   Current       64 non-null     object
 5   Return        64 non-null     object
dtypes: object(6)
memory usage: 3.1+ KB


In [13]:
# "stacking dataframes"
# pd.concat() is used to concatenate the DataFrames vertically.
# The ignore_index=True parameter ensures that the resulting DataFrame has a new index,
# ignoring the original indices of the input DataFrames.
# The stacked_df now contains the concatenated DataFrame.
stacked_ipos_df = pd.concat([df_ipos_2024, df_ipos_2023], ignore_index=True)
stacked_ipos_df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$28.94,20.58%
1,"Apr 26, 2024",ZONE,"CleanCore Solutions, Inc.",$4.00,$3.18,-21.50%
2,"Apr 25, 2024",RBRK,"Rubrik, Inc.",$32.00,$34.87,8.97%
3,"Apr 25, 2024",LOAR,Loar Holdings Inc.,$28.00,$51.10,82.50%
4,"Apr 25, 2024",MRX,Marex Group plc,$19.00,$19.15,0.79%
...,...,...,...,...,...,...
213,"Jan 25, 2023",QSG,QuantaSing Group Ltd,$12.50,$3.20,-74.48%
214,"Jan 20, 2023",CVKD,"Cadrenal Therapeutics, Inc.",$5.00,$0.50,-90.00%
215,"Jan 13, 2023",SKWD,"Skyward Specialty Insurance Group, Inc.",$15.00,$37.60,150.67%
216,"Jan 13, 2023",ISRL,Israel Acquisitions Corp,$10.00,$10.92,9.20%


In [14]:
# Convert the 'IPO Date' to datetime() 
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'],format='mixed')

# Convert the 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs).
#ipos_filings_df['Shares Offered'] = ipos_filings_df['Shares Offered'].str.replace('-', 'nan').astype(float)
stacked_ipos_df.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2024-05-01,VIK,Viking Holdings Ltd.,$24.00,$28.94,20.58%
1,2024-04-26,ZONE,"CleanCore Solutions, Inc.",$4.00,$3.18,-21.50%
2,2024-04-25,RBRK,"Rubrik, Inc.",$32.00,$34.87,8.97%
3,2024-04-25,LOAR,Loar Holdings Inc.,$28.00,$51.10,82.50%
4,2024-04-25,MRX,Marex Group plc,$19.00,$19.15,0.79%


In [15]:
# filter the stacked_ipos_df to 'IPO Date' prior to 2024-03-01
stacked_ipos_df_filtered=stacked_ipos_df.loc[stacked_ipos_df['IPO Date']<"2024-03-01"]       
stacked_ipos_df_filtered.count()

IPO Date        185
Symbol          185
Company Name    185
IPO Price       185
Current         185
Return          185
dtype: int64

In [16]:
stock_lst=list(stacked_ipos_df_filtered['Symbol'])
stock_lst.remove('RYZB')
stock_lst.remove('LEGT')
stock_lst.remove('JVSA')
stock_lst.remove('DYCQ')
stock_lst.remove('PTHR')
stock_lst.append('PTHRF')
stock_lst.append('JVSAU')
stock_lst.append('DYCQU')
stock_lst.append('LEGT-UN')
stock_lst

['SMXT',
 'VHAI',
 'CHRO',
 'UMAC',
 'TBBB',
 'MGX',
 'HLXB',
 'TELO',
 'KYTX',
 'PMNT',
 'AHR',
 'ANRO',
 'GUTS',
 'AS',
 'FBLG',
 'BTSG',
 'AVBP',
 'HAO',
 'CGON',
 'YIBO',
 'SUGP',
 'JL',
 'KSPI',
 'PSBD',
 'CCTG',
 'SYNX',
 'SDHC',
 'ROMA',
 'IROH',
 'LGCB',
 'ZKH',
 'BAYA',
 'INHD',
 'AFJK',
 'GSIW',
 'FEBO',
 'CLBR',
 'ELAB',
 'RR',
 'DDC',
 'SHIM',
 'GLAC',
 'SGN',
 'HG',
 'CRGX',
 'ANSC',
 'AITR',
 'GVH',
 'LXEO',
 'PAPL',
 'ATGL',
 'MNR',
 'WBUY',
 'NCL',
 'BIRK',
 'GMM',
 'PMEC',
 'LRHC',
 'GPAK',
 'SPKL',
 'QETA',
 'MSS',
 'ANL',
 'SYRA',
 'VSME',
 'LRE',
 'TURB',
 'MDBH',
 'KVYO',
 'CART',
 'DTCK',
 'NMRA',
 'ARM',
 'SPPL',
 'NWGL',
 'SWIN',
 'IVP',
 'NNAG',
 'SRM',
 'SPGC',
 'LQR',
 'NRXS',
 'FTEL',
 'MIRA',
 'PXDT',
 'CTNT',
 'HRYU',
 'SRFM',
 'PRZO',
 'HYAC',
 'KVAC',
 'JNVR',
 'ELWS',
 'WRNT',
 'TSBX',
 'ODD',
 'APGE',
 'NETD',
 'SGMT',
 'BOWN',
 'SXTP',
 'PWM',
 'VTMX',
 'INTS',
 'SVV',
 'KGS',
 'FIHL',
 'GENK',
 'BUJA',
 'BOF',
 'AZTR',
 'CAVA',
 'ESHA',
 'ATMU',
 'AT

In [17]:
len(stock_lst)

184

In [18]:
end_dt = date(2024,3,1)
df_OHLC=yf.download(stock_lst,end=end_dt,progress=True)
df_OHLC

[*********************100%%**********************]  184 of 184 completed


Price      Adj Close                                                     \
Ticker          AACT       AESI    AFJK        AHR    AITR  AIXI   ALCY   
Date                                                                      
2009-12-29       NaN        NaN     NaN        NaN     NaN   NaN    NaN   
2009-12-30       NaN        NaN     NaN        NaN     NaN   NaN    NaN   
2009-12-31       NaN        NaN     NaN        NaN     NaN   NaN    NaN   
2010-01-04       NaN        NaN     NaN        NaN     NaN   NaN    NaN   
2010-01-05       NaN        NaN     NaN        NaN     NaN   NaN    NaN   
...              ...        ...     ...        ...     ...   ...    ...   
2024-02-23     10.55  18.320000  10.175  13.365355  10.240  1.87  10.55   
2024-02-26     10.54  18.330000  10.180  13.031221  10.240  1.90  10.56   
2024-02-27     10.55  19.809999  10.174  13.316218  10.242  2.43  10.56   
2024-02-28     10.48  19.299999  10.180  13.404666  10.240  2.06  10.54   
2024-02-29     10.53  18.860001  10.154  13.434148  10.240  1.98  10.55   

Price                            ...      Volume                      \
Ticker        ANL   ANRO   ANSC  ...        VHAI      VSME      VTMX   
Date                             ...                                   
2009-12-29    NaN    NaN    NaN  ...         NaN       NaN       NaN   
2009-12-30    NaN    NaN    NaN  ...         NaN       NaN       NaN   
2009-12-31    NaN    NaN    NaN  ...         NaN       NaN       NaN   
2010-01-04    NaN    NaN    NaN  ...         NaN       NaN       NaN   
2010-01-05    NaN    NaN    NaN  ...         NaN       NaN       NaN   
...           ...    ...    ...  ...         ...       ...       ...   
2024-02-23  7.151  13.72  10.14  ...         NaN  166900.0  125000.0   
2024-02-26  7.150  15.31  10.15  ...   2444600.0  326800.0   77300.0   
2024-02-27  7.150  14.36  10.15  ...  10196900.0  963200.0  114200.0   
2024-02-28  7.150  15.33  10.13  ...   1554300.0  393900.0  165400.0   
2024-02-29  7.150  15.40  10.14  ...   1853300.0  177900.0   90100.0   

Price                                                                        
Ticker          WBUY       WLGS      WRNT YGFGF     YIBO      ZJYL      ZKH  
Date                                                                         
2009-12-29       NaN        NaN       NaN   NaN      NaN       NaN      NaN  
2009-12-30       NaN        NaN       NaN   NaN      NaN       NaN      NaN  
2009-12-31       NaN        NaN       NaN   NaN      NaN       NaN      NaN  
2010-01-04       NaN        NaN       NaN   NaN      NaN       NaN      NaN  
2010-01-05       NaN        NaN       NaN   NaN      NaN       NaN      NaN  
...              ...        ...       ...   ...      ...       ...      ...  
2024-02-23  225200.0   107100.0  128200.0   0.0  35700.0  323200.0  54100.0  
2024-02-26  137400.0  1060100.0   38900.0   0.0  20400.0  578500.0  42700.0  
2024-02-27  161600.0   375400.0  147600.0   0.0  19300.0  218900.0  40000.0  
2024-02-28  182900.0   266700.0  187200.0   0.0   9700.0  200900.0  55200.0  
2024-02-29   88600.0   139100.0   62700.0   0.0  11400.0  120300.0  76600.0  

[3566 rows x 1104 columns]

In [19]:
df=df_OHLC.copy()
df=df['Adj Close']
df.tail(30)

Ticker,AACT,AESI,AFJK,AHR,AITR,AIXI,ALCY,ANL,ANRO,ANSC,...,VHAI,VSME,VTMX,WBUY,WLGS,WRNT,YGFGF,YIBO,ZJYL,ZKH
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-18,10.465,16.587967,NaN,NaN,10.200,1.79,10.490,9.050,NaN,10.10,...,NaN,0.369,36.618332,0.399,0.700,0.350,1.530,NaN,5.2575,16.730000
2024-01-19,10.470,16.172773,NaN,NaN,10.210,1.61,10.490,8.989,NaN,10.11,...,NaN,0.330,37.434284,0.380,0.584,0.333,1.570,NaN,5.2500,16.100000
2024-01-22,10.480,16.933960,NaN,NaN,10.210,1.58,10.500,9.120,NaN,10.13,...,NaN,0.319,37.334778,0.395,0.590,0.330,1.420,NaN,5.3500,16.200001
2024-01-23,10.480,16.775791,10.160,NaN,10.210,1.69,10.600,9.120,NaN,10.13,...,NaN,0.328,37.205418,0.450,0.610,0.350,1.270,NaN,8.3450,16.809999
2024-01-24,10.480,16.627508,10.160,NaN,10.200,1.82,10.510,9.213,NaN,10.12,...,NaN,0.649,37.444237,0.460,0.600,0.348,1.250,NaN,10.5925,18.520000
2024-01-25,10.480,17.072359,10.170,NaN,10.200,1.88,10.560,9.213,NaN,10.12,...,NaN,0.487,38.568653,0.467,0.620,0.350,1.060,2.790,13.8055,19.040001
2024-01-26,10.490,17.240414,10.170,NaN,10.210,1.90,10.560,9.213,NaN,10.12,...,NaN,0.650,38.528854,0.445,0.660,0.345,0.858,2.720,8.9000,17.150000
2024-01-29,10.500,17.368925,10.180,NaN,10.210,1.90,10.520,9.130,NaN,10.12,...,NaN,0.599,38.260185,0.464,0.659,0.344,0.685,2.600,9.1495,17.750000
2024-01-30,10.490,17.527094,10.170,NaN,10.200,1.94,10.525,9.130,NaN,10.13,...,NaN,0.478,37.941769,0.428,0.790,0.330,0.731,2.660,10.5200,18.120001


In [20]:
df_tmp = pd.DataFrame(columns=[f'growth_future_{d}d' for d in range(1,31)],index=df.columns)
df_tmp

,growth_future_1d,growth_future_2d,growth_future_3d,growth_future_4d,growth_future_5d,growth_future_6d,growth_future_7d,growth_future_8d,growth_future_9d,growth_future_10d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
Ticker,,,,,,,,,,,,,,,,,,,,,
AACT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AESI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFJK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AITR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WRNT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YGFGF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YIBO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Function to calculate future growth

In [21]:
def calc_growth_futre_xd(qty_days:int,stock:str,df_stocks:pd.DataFrame)->float:
  df_temp2=df.loc[df[stock].isna()==False,stock].copy()
  try:
    return (df_temp2.shift(-qty_days)/df_temp2).dropna().iloc[0]
  except Exception as e :
    return np.NaN

stock_lst.sort()

t1= time.time()
for ticket in stock_lst:
  for d in range(1,31):
    df_tmp.loc[ticket,f"growth_future_{d}d"]=calc_growth_futre_xd(d,ticket,df)
for c in df_tmp.columns:
  df_tmp[c]=  pd.to_numeric(df_tmp[c],errors='coerce')
# df_tmp.info()
# df_tmp.describe()
print(f"time enlapsed {time.time()-t1:.2f}")
# df_tmp.loc['AESI'][f"growth_future_{1}d"]=1
df_tmp

time enlapsed 5.88


,growth_future_1d,growth_future_2d,growth_future_3d,growth_future_4d,growth_future_5d,growth_future_6d,growth_future_7d,growth_future_8d,growth_future_9d,growth_future_10d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
Ticker,,,,,,,,,,,,,,,,,,,,,
AACT,0.999011,1.000000,1.000000,1.000989,1.000989,1.000989,1.002967,1.002967,1.003956,1.004946,...,1.005539,1.004946,1.004946,1.004946,1.005935,1.005935,1.005935,1.007913,1.007913,1.007913
AESI,0.973451,0.943363,0.979941,0.946903,0.951032,0.939823,0.949853,0.965192,0.968142,0.943953,...,1.023009,1.049557,1.050737,1.058997,1.076106,1.074926,1.057817,1.066077,1.056637,1.072566
AFJK,1.000000,1.000984,1.000984,1.001969,1.000984,1.001969,1.000984,1.000984,1.000492,1.000984,...,1.001969,1.001476,1.001969,1.001378,1.001969,0.999409,NaN,NaN,NaN,NaN
AHR,0.987897,0.992436,0.994705,1.009834,1.040847,1.031770,1.049168,1.014372,1.013616,1.024962,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AITR,1.000986,1.003945,1.003945,1.004043,1.004931,1.004931,1.004931,1.005424,1.004931,1.005917,...,1.006903,1.006903,1.006903,1.007890,1.008876,1.007890,1.007692,1.007692,1.008383,1.008876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WRNT,0.788372,0.572093,0.508139,0.441860,0.372093,0.402326,0.369767,0.365116,0.318605,0.313953,...,0.241860,0.223256,0.226744,0.209302,0.244186,0.217674,0.209302,0.209302,0.213953,0.225814
YGFGF,0.975064,0.976864,0.997686,1.002571,0.940874,0.866324,0.843188,0.843188,0.832905,0.858612,...,0.950386,0.904884,1.002571,0.866324,0.868895,0.750643,0.758355,0.754499,0.830334,0.943445
YIBO,0.974910,0.931900,0.953405,0.867384,0.821505,0.737993,0.727599,0.788530,0.770609,0.770609,...,0.931900,0.982079,1.014337,0.982079,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_tmp.describe()

,growth_future_1d,growth_future_2d,growth_future_3d,growth_future_4d,growth_future_5d,growth_future_6d,growth_future_7d,growth_future_8d,growth_future_9d,growth_future_10d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
count,184.000000,184.000000,183.000000,182.000000,182.000000,182.000000,181.000000,181.000000,179.000000,179.000000,...,166.000000,166.000000,166.000000,163.000000,161.000000,159.000000,158.000000,158.000000,156.000000,154.000000
mean,0.945810,0.937106,0.932026,0.921486,0.914781,0.907470,0.897106,0.897288,0.898583,0.895306,...,0.916542,0.908094,0.908017,0.905103,0.893409,0.899677,0.933847,0.939195,0.937890,0.939298
std,0.170227,0.205919,0.247176,0.264042,0.295091,0.309842,0.298984,0.320826,0.353696,0.367094,...,0.529683,0.487513,0.493698,0.502190,0.498237,0.544435,0.831805,0.833095,0.835257,0.828967
min,0.153569,0.108733,0.086641,0.094257,0.081118,0.087677,0.085261,0.090123,0.095616,0.096997,...,0.048326,0.048326,0.049361,0.048326,0.048326,0.045219,0.044529,0.046945,0.042803,0.040387
25%,0.921392,0.876712,0.853571,0.835599,0.792430,0.784618,0.790333,0.750000,0.731103,0.711589,...,0.625299,0.616880,0.616976,0.633517,0.607884,0.606017,0.602782,0.626498,0.610721,0.623529
50%,1.000000,0.999550,0.997959,0.995516,0.996965,0.976002,0.980443,0.981308,0.987915,0.978735,...,0.977389,0.985499,0.984442,0.977778,0.966194,0.977839,0.973983,0.973220,0.968098,0.984353
75%,1.011792,1.017794,1.009769,1.009530,1.009926,1.007620,1.006897,1.008815,1.011662,1.014198,...,1.011947,1.022487,1.020213,1.021537,1.016716,1.023933,1.021156,1.025557,1.024041,1.020630
max,1.362069,1.464015,2.380000,2.083710,2.262443,2.529870,2.173913,2.359740,2.751948,3.176087,...,4.500000,3.871041,3.846154,3.803394,3.427273,4.817886,9.056122,9.081632,9.265306,9.372449


In [23]:
quantil_75=df_tmp.describe().loc['75%']
quantil_75

growth_future_1d     1.011792
growth_future_2d     1.017794
growth_future_3d     1.009769
growth_future_4d     1.009530
growth_future_5d     1.009926
growth_future_6d     1.007620
growth_future_7d     1.006897
growth_future_8d     1.008815
growth_future_9d     1.011662
growth_future_10d    1.014198
growth_future_11d    1.013850
growth_future_12d    1.019350
growth_future_13d    1.020168
growth_future_14d    1.014986
growth_future_15d    1.014078
growth_future_16d    1.013298
growth_future_17d    1.010832
growth_future_18d    1.011820
growth_future_19d    1.014823
growth_future_20d    1.014142
growth_future_21d    1.011947
growth_future_22d    1.022487
growth_future_23d    1.020213
growth_future_24d    1.021537
growth_future_25d    1.016716
growth_future_26d    1.023933
growth_future_27d    1.021156
growth_future_28d    1.025557
growth_future_29d    1.024041
growth_future_30d    1.020630
Name: 75%, dtype: float64

### Question 2: Answer

In [24]:
# Q2 Answer
days=f"{quantil_75[quantil_75.max()==quantil_75].index[0]}"
days=days.replace("growth_future_","").replace('d','')
print(f"the optimal number of days is {days}")

the optimal number of days is 28


---
### Question 3: Is Growth Concentrated in the Largest Stocks?

**Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**

Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks 
for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

`US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']`

`EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']`

`INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']`

`LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS`
<br/>

Now let's add the top 12-22 stocks (as of end-April 2024):
<br/>

`NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']`

`NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']`

`NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']`

`LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA`



In [59]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

# Now let's add the top 12-22 stocks (as of end-April 2024):

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA



In [38]:
ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
ALL_TICKERS

['MSFT',
 'AAPL',
 'GOOG',
 'NVDA',
 'AMZN',
 'META',
 'BRK-B',
 'LLY',
 'AVGO',
 'V',
 'JPM',
 'NVO',
 'MC.PA',
 'ASML',
 'RMS.PA',
 'OR.PA',
 'SAP',
 'ACN',
 'TTE',
 'SIE.DE',
 'IDEXY',
 'CDI.PA',
 'RELIANCE.NS',
 'TCS.NS',
 'HDB',
 'BHARTIARTL.NS',
 'IBN',
 'SBIN.NS',
 'LICI.NS',
 'INFY',
 'ITC.NS',
 'HINDUNILVR.NS',
 'LT.NS']

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)

Calculate  `growth_7d` for every stock and every day.
Get the average daily `growth_7d` for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:
| Date   |      ticker_category      |  growth_7d |
|----------|:-------------:|------:|
| 2014-01-01 |  LARGE | 1.011684 |
| 2014-01-01 |   LARGEST   |   1.011797 |

On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)

In [42]:
import time

# Define an empty dataframe to store stock data 
stocks_df = pd.DataFrame({'A' : []})

# Loop through list of ticker symbols
for i,ticker in enumerate(ALL_TICKERS):
  
  # Print ticker name for progress tracking
  print(i,ticker)

  # Download historical price data for ticker from Yahoo Finance
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns... calculate daily, weekly, monthly returns over periods
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict, Label rows where future 5d return is positive
  historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)

 # Append data to master dataframe
  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed


32 LT.NS


[*********************100%%**********************]  1 of 1 completed


In [43]:
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'


In [44]:
stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS))

In [46]:
# count of observations between US-EU-INDIA stocks
stocks_df.ticker_type.value_counts()

ticker_type
US       80717
EU       77058
INDIA    63334
Name: count, dtype: int64

In [48]:
# unique tickers
stocks_df.Ticker.nunique()

33

In [50]:
# count of observations by stock
stocks_df.Ticker.value_counts()

Ticker
LLY              13094
JPM              11128
AAPL             10940
NVO              10845
MSFT              9614
CDI.PA            8327
TTE               8192
ASML              7337
SAP               7208
HINDUNILVR.NS     7121
SBIN.NS           7119
RELIANCE.NS       7118
ITC.NS            7118
BRK-B             7045
SIE.DE            7045
AMZN              6788
NVDA              6363
INFY              6330
MC.PA             6257
RMS.PA            6257
OR.PA             6257
IBN               6065
ACN               5735
HDB               5734
LT.NS             5426
BHARTIARTL.NS     5423
TCS.NS            5394
GOOG              4962
V                 4061
AVGO              3712
IDEXY             3598
META              3010
LICI.NS            486
Name: count, dtype: int64

In [51]:
stocks_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
AAPL,US,1980-12-12,2024-05-06
ACN,EU,2001-07-19,2024-05-06
AMZN,US,1997-05-15,2024-05-06
ASML,EU,1995-03-15,2024-05-06
AVGO,US,2009-08-06,2024-05-06
BHARTIARTL.NS,INDIA,2002-07-01,2024-05-06
BRK-B,US,1996-05-09,2024-05-06
CDI.PA,EU,1992-01-27,2024-05-06
GOOG,US,2004-08-19,2024-05-06


In [52]:
#  average growth 365days
stocks_df[stocks_df.Year>=2020].groupby(by=['Year','ticker_type']).growth_365d.mean()

Year  ticker_type
2020  EU             1.245388
      INDIA          1.101896
      US             1.484070
2021  EU             1.582170
      INDIA          1.546424
      US             1.797571
2022  EU             1.191169
      INDIA          1.379889
      US             1.182379
2023  EU             1.246533
      INDIA          1.201110
      US             1.203253
2024  EU             1.640018
      INDIA          1.302174
      US             2.160563
Name: growth_365d, dtype: float64

In [53]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

In [54]:
# filtering only on stats after 2000
stocks_df[stocks_df.Date>='2000-01-01'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182642 entries, 3490 to 221108
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Open                          182642 non-null  float64       
 1   High                          182642 non-null  float64       
 2   Low                           182642 non-null  float64       
 3   Close                         182642 non-null  float64       
 4   Adj Close                     182642 non-null  float64       
 5   Volume                        182642 non-null  int64         
 6   Ticker                        182642 non-null  object        
 7   Year                          182642 non-null  int32         
 8   Month                         182642 non-null  int32         
 9   Weekday                       182642 non-null  int32         
 10  Date                          182642 non-null  datetime64[ns]
 11  growth_1d      

In [55]:
stocks_df[stocks_df.Date=='2024-04-01'].tail()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_90d,growth_365d,growth_future_5d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_5d_future,ticker_type
195091,925.000000,984.450012,924.000000,970.599976,970.599976,4863536,LICI.NS,2024,4,0,...,1.599113,1.556710,1.026891,906.085004,953.950003,0,0.062281,1034.237352,1,INDIA
201418,17.889999,17.920000,17.500000,17.580000,17.580000,14601600,INFY,2024,4,0,...,1.003998,1.024508,1.015927,18.156000,18.777500,0,0.023891,13.940634,1,INDIA
208539,429.100006,431.350006,425.750000,426.700012,426.700012,9382308,ITC.NS,2024,4,0,...,0.987044,1.314436,1.005625,422.235004,416.537502,1,0.013124,130.246651,1,INDIA
215660,2270.000000,2295.000000,2262.000000,2285.899902,2285.899902,1427873,HINDUNILVR.NS,2024,4,0,...,0.917590,0.894050,0.992585,2266.539990,2327.240015,0,0.014436,1049.970071,0,INDIA
221086,3780.850098,3851.850098,3780.850098,3838.000000,3838.000000,1905190,LT.NS,2024,4,0,...,1.257886,2.010850,0.992144,3626.594971,3618.109985,1,0.018499,2188.619806,0,INDIA


In [56]:
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'


In [ ]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

In [ ]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

In [ ]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

In [ ]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

In [ ]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

In [ ]:
ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
ALL_TICKERS

['MSFT',
 'AAPL',
 'GOOG',
 'NVDA',
 'AMZN',
 'META',
 'BRK-B',
 'LLY',
 'AVGO',
 'V',
 'JPM',
 'NVO',
 'MC.PA',
 'ASML',
 'RMS.PA',
 'OR.PA',
 'SAP',
 'ACN',
 'TTE',
 'SIE.DE',
 'IDEXY',
 'CDI.PA',
 'RELIANCE.NS',
 'TCS.NS',
 'HDB',
 'BHARTIARTL.NS',
 'IBN',
 'SBIN.NS',
 'LICI.NS',
 'INFY',
 'ITC.NS',
 'HINDUNILVR.NS',
 'LT.NS']

In [ ]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed


2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed


32 LT.NS


[*********************100%%**********************]  1 of 1 completed


In [ ]:
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'


In [ ]:
stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS))

In [ ]:
# count of observations between US-EU-INDIA stocks
stocks_df.ticker_type.value_counts()

ticker_type
US       80639
EU       76986
INDIA    63265
Name: count, dtype: int64

In [ ]:
# unique tickers
stocks_df.Ticker.nunique()

33

In [ ]:
# count of observations by stock
stocks_df.Ticker.value_counts()

Ticker
LLY              13087
JPM              11121
AAPL             10933
NVO              10838
MSFT              9607
CDI.PA            8321
TTE               8185
ASML              7330
SAP               7201
HINDUNILVR.NS     7115
SBIN.NS           7113
RELIANCE.NS       7112
ITC.NS            7112
SIE.DE            7039
BRK-B             7038
AMZN              6780
NVDA              6356
INFY              6323
MC.PA             6251
RMS.PA            6251
OR.PA             6251
IBN               6058
ACN               5728
HDB               5727
LT.NS             5420
BHARTIARTL.NS     5417
TCS.NS            5388
GOOG              4955
V                 4054
AVGO              3705
IDEXY             3591
META              3003
LICI.NS            480
Name: count, dtype: int64

In [ ]:
stocks_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
AAPL,US,1980-12-12,2024-04-25
ACN,EU,2001-07-19,2024-04-25
AMZN,US,1997-05-15,2024-04-24
ASML,EU,1995-03-15,2024-04-25
AVGO,US,2009-08-06,2024-04-25
BHARTIARTL.NS,INDIA,2002-07-01,2024-04-25
BRK-B,US,1996-05-09,2024-04-25
CDI.PA,EU,1992-01-27,2024-04-25
GOOG,US,2004-08-19,2024-04-25


In [ ]:
#  average growth 365days
stocks_df[stocks_df.Year>=2020].groupby(by=['Year','ticker_type']).growth_365d.mean()

Year  ticker_type
2020  EU             1.245388
      INDIA          1.101896
      US             1.484070
2021  EU             1.582170
      INDIA          1.546424
      US             1.797571
2022  EU             1.191169
      INDIA          1.379889
      US             1.182379
2023  EU             1.246533
      INDIA          1.201110
      US             1.203253
2024  EU             1.650721
      INDIA          1.305762
      US             2.153187
Name: growth_365d, dtype: float64

In [ ]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

In [ ]:
# filtering only on stats after 2000
stocks_df[stocks_df.Date>='2000-01-01'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182423 entries, 3490 to 220889
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Open                          182423 non-null  float64       
 1   High                          182423 non-null  float64       
 2   Low                           182423 non-null  float64       
 3   Close                         182423 non-null  float64       
 4   Adj Close                     182423 non-null  float64       
 5   Volume                        182423 non-null  int64         
 6   Ticker                        182423 non-null  object        
 7   Year                          182423 non-null  int32         
 8   Month                         182423 non-null  int32         
 9   Weekday                       182423 non-null  int32         
 10  Date                          182423 non-null  datetime64[ns]
 11  growth_1d      

In [ ]:
stocks_df[stocks_df.Date=='2024-04-01'].tail()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_90d,growth_365d,growth_future_5d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_5d_future,ticker_type
194903,925.000000,984.450012,924.000000,970.599976,970.599976,4863536,LICI.NS,2024,4,0,...,1.599113,1.556710,1.026891,906.085004,953.950003,0,0.062281,1034.237352,1,INDIA
201224,17.889999,17.920000,17.500000,17.580000,17.580000,14601600,INFY,2024,4,0,...,1.003998,1.024508,1.015927,18.156000,18.777500,0,0.023891,13.940634,1,INDIA
208338,429.100006,431.350006,425.750000,426.700012,426.700012,9382308,ITC.NS,2024,4,0,...,0.987044,1.314436,1.005625,422.235004,416.537502,1,0.013124,130.246651,1,INDIA
215453,2270.000000,2295.000000,2262.000000,2285.899902,2285.899902,1427873,HINDUNILVR.NS,2024,4,0,...,0.917590,0.894051,0.992585,2266.539990,2327.240015,0,0.014436,1049.970071,0,INDIA
220873,3780.850098,3851.850098,3780.850098,3838.000000,3838.000000,1905190,LT.NS,2024,4,0,...,1.257886,2.010850,0.992144,3626.594971,3618.109985,1,0.018499,2188.619806,0,INDIA


In [ ]:
ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
ALL_TICKERS

['MSFT',
 'AAPL',
 'GOOG',
 'NVDA',
 'AMZN',
 'META',
 'BRK-B',
 'LLY',
 'AVGO',
 'V',
 'JPM',
 'NVO',
 'MC.PA',
 'ASML',
 'RMS.PA',
 'OR.PA',
 'SAP',
 'ACN',
 'TTE',
 'SIE.DE',
 'IDEXY',
 'CDI.PA',
 'RELIANCE.NS',
 'TCS.NS',
 'HDB',
 'BHARTIARTL.NS',
 'IBN',
 'SBIN.NS',
 'LICI.NS',
 'INFY',
 'ITC.NS',
 'HINDUNILVR.NS',
 'LT.NS']

In [ ]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed


2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed


32 LT.NS


[*********************100%%**********************]  1 of 1 completed


In [ ]:
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'


In [ ]:
stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS))

In [ ]:
# count of observations between US-EU-INDIA stocks
stocks_df.ticker_type.value_counts()

ticker_type
US       80639
EU       76986
INDIA    63265
Name: count, dtype: int64

In [ ]:
# unique tickers
stocks_df.Ticker.nunique()

33

In [ ]:
# count of observations by stock
stocks_df.Ticker.value_counts()

Ticker
LLY              13087
JPM              11121
AAPL             10933
NVO              10838
MSFT              9607
CDI.PA            8321
TTE               8185
ASML              7330
SAP               7201
HINDUNILVR.NS     7115
SBIN.NS           7113
RELIANCE.NS       7112
ITC.NS            7112
SIE.DE            7039
BRK-B             7038
AMZN              6780
NVDA              6356
INFY              6323
MC.PA             6251
RMS.PA            6251
OR.PA             6251
IBN               6058
ACN               5728
HDB               5727
LT.NS             5420
BHARTIARTL.NS     5417
TCS.NS            5388
GOOG              4955
V                 4054
AVGO              3705
IDEXY             3591
META              3003
LICI.NS            480
Name: count, dtype: int64

In [ ]:
stocks_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
AAPL,US,1980-12-12,2024-04-25
ACN,EU,2001-07-19,2024-04-25
AMZN,US,1997-05-15,2024-04-24
ASML,EU,1995-03-15,2024-04-25
AVGO,US,2009-08-06,2024-04-25
BHARTIARTL.NS,INDIA,2002-07-01,2024-04-25
BRK-B,US,1996-05-09,2024-04-25
CDI.PA,EU,1992-01-27,2024-04-25
GOOG,US,2004-08-19,2024-04-25


In [ ]:
#  average growth 365days
stocks_df[stocks_df.Year>=2020].groupby(by=['Year','ticker_type']).growth_365d.mean()

Year  ticker_type
2020  EU             1.245388
      INDIA          1.101896
      US             1.484070
2021  EU             1.582170
      INDIA          1.546424
      US             1.797571
2022  EU             1.191169
      INDIA          1.379889
      US             1.182379
2023  EU             1.246533
      INDIA          1.201110
      US             1.203253
2024  EU             1.650721
      INDIA          1.305762
      US             2.153187
Name: growth_365d, dtype: float64

In [ ]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

In [ ]:
# filtering only on stats after 2000
stocks_df[stocks_df.Date>='2000-01-01'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182423 entries, 3490 to 220889
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Open                          182423 non-null  float64       
 1   High                          182423 non-null  float64       
 2   Low                           182423 non-null  float64       
 3   Close                         182423 non-null  float64       
 4   Adj Close                     182423 non-null  float64       
 5   Volume                        182423 non-null  int64         
 6   Ticker                        182423 non-null  object        
 7   Year                          182423 non-null  int32         
 8   Month                         182423 non-null  int32         
 9   Weekday                       182423 non-null  int32         
 10  Date                          182423 non-null  datetime64[ns]
 11  growth_1d      

In [ ]:
stocks_df[stocks_df.Date=='2024-04-01'].tail()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_90d,growth_365d,growth_future_5d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_5d_future,ticker_type
194903,925.000000,984.450012,924.000000,970.599976,970.599976,4863536,LICI.NS,2024,4,0,...,1.599113,1.556710,1.026891,906.085004,953.950003,0,0.062281,1034.237352,1,INDIA
201224,17.889999,17.920000,17.500000,17.580000,17.580000,14601600,INFY,2024,4,0,...,1.003998,1.024508,1.015927,18.156000,18.777500,0,0.023891,13.940634,1,INDIA
208338,429.100006,431.350006,425.750000,426.700012,426.700012,9382308,ITC.NS,2024,4,0,...,0.987044,1.314436,1.005625,422.235004,416.537502,1,0.013124,130.246651,1,INDIA
215453,2270.000000,2295.000000,2262.000000,2285.899902,2285.899902,1427873,HINDUNILVR.NS,2024,4,0,...,0.917590,0.894051,0.992585,2266.539990,2327.240015,0,0.014436,1049.970071,0,INDIA
220873,3780.850098,3851.850098,3780.850098,3838.000000,3838.000000,1905190,LT.NS,2024,4,0,...,1.257886,2.010850,0.992144,3626.594971,3618.109985,1,0.018499,2188.619806,0,INDIA


In [ ]:
ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
ALL_TICKERS

['MSFT',
 'AAPL',
 'GOOG',
 'NVDA',
 'AMZN',
 'META',
 'BRK-B',
 'LLY',
 'AVGO',
 'V',
 'JPM',
 'NVO',
 'MC.PA',
 'ASML',
 'RMS.PA',
 'OR.PA',
 'SAP',
 'ACN',
 'TTE',
 'SIE.DE',
 'IDEXY',
 'CDI.PA',
 'RELIANCE.NS',
 'TCS.NS',
 'HDB',
 'BHARTIARTL.NS',
 'IBN',
 'SBIN.NS',
 'LICI.NS',
 'INFY',
 'ITC.NS',
 'HINDUNILVR.NS',
 'LT.NS']

In [ ]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed


2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed


32 LT.NS


[*********************100%%**********************]  1 of 1 completed


In [ ]:
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'


In [ ]:
stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS))

In [ ]:
# count of observations between US-EU-INDIA stocks
stocks_df.ticker_type.value_counts()

ticker_type
US       80639
EU       76986
INDIA    63265
Name: count, dtype: int64

In [ ]:
# unique tickers
stocks_df.Ticker.nunique()

33

In [ ]:
# count of observations by stock
stocks_df.Ticker.value_counts()

Ticker
LLY              13087
JPM              11121
AAPL             10933
NVO              10838
MSFT              9607
CDI.PA            8321
TTE               8185
ASML              7330
SAP               7201
HINDUNILVR.NS     7115
SBIN.NS           7113
RELIANCE.NS       7112
ITC.NS            7112
SIE.DE            7039
BRK-B             7038
AMZN              6780
NVDA              6356
INFY              6323
MC.PA             6251
RMS.PA            6251
OR.PA             6251
IBN               6058
ACN               5728
HDB               5727
LT.NS             5420
BHARTIARTL.NS     5417
TCS.NS            5388
GOOG              4955
V                 4054
AVGO              3705
IDEXY             3591
META              3003
LICI.NS            480
Name: count, dtype: int64

In [ ]:
stocks_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
AAPL,US,1980-12-12,2024-04-25
ACN,EU,2001-07-19,2024-04-25
AMZN,US,1997-05-15,2024-04-24
ASML,EU,1995-03-15,2024-04-25
AVGO,US,2009-08-06,2024-04-25
BHARTIARTL.NS,INDIA,2002-07-01,2024-04-25
BRK-B,US,1996-05-09,2024-04-25
CDI.PA,EU,1992-01-27,2024-04-25
GOOG,US,2004-08-19,2024-04-25


In [ ]:
#  average growth 365days
stocks_df[stocks_df.Year>=2020].groupby(by=['Year','ticker_type']).growth_365d.mean()

Year  ticker_type
2020  EU             1.245388
      INDIA          1.101896
      US             1.484070
2021  EU             1.582170
      INDIA          1.546424
      US             1.797571
2022  EU             1.191169
      INDIA          1.379889
      US             1.182379
2023  EU             1.246533
      INDIA          1.201110
      US             1.203253
2024  EU             1.650721
      INDIA          1.305762
      US             2.153187
Name: growth_365d, dtype: float64

In [ ]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

In [ ]:
# filtering only on stats after 2000
stocks_df[stocks_df.Date>='2000-01-01'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182423 entries, 3490 to 220889
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Open                          182423 non-null  float64       
 1   High                          182423 non-null  float64       
 2   Low                           182423 non-null  float64       
 3   Close                         182423 non-null  float64       
 4   Adj Close                     182423 non-null  float64       
 5   Volume                        182423 non-null  int64         
 6   Ticker                        182423 non-null  object        
 7   Year                          182423 non-null  int32         
 8   Month                         182423 non-null  int32         
 9   Weekday                       182423 non-null  int32         
 10  Date                          182423 non-null  datetime64[ns]
 11  growth_1d      

In [ ]:
stocks_df[stocks_df.Date=='2024-04-01'].tail()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_90d,growth_365d,growth_future_5d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_5d_future,ticker_type
194903,925.000000,984.450012,924.000000,970.599976,970.599976,4863536,LICI.NS,2024,4,0,...,1.599113,1.556710,1.026891,906.085004,953.950003,0,0.062281,1034.237352,1,INDIA
201224,17.889999,17.920000,17.500000,17.580000,17.580000,14601600,INFY,2024,4,0,...,1.003998,1.024508,1.015927,18.156000,18.777500,0,0.023891,13.940634,1,INDIA
208338,429.100006,431.350006,425.750000,426.700012,426.700012,9382308,ITC.NS,2024,4,0,...,0.987044,1.314436,1.005625,422.235004,416.537502,1,0.013124,130.246651,1,INDIA
215453,2270.000000,2295.000000,2262.000000,2285.899902,2285.899902,1427873,HINDUNILVR.NS,2024,4,0,...,0.917590,0.894051,0.992585,2266.539990,2327.240015,0,0.014436,1049.970071,0,INDIA
220873,3780.850098,3851.850098,3780.850098,3838.000000,3838.000000,1905190,LT.NS,2024,4,0,...,1.257886,2.010850,0.992144,3626.594971,3618.109985,1,0.018499,2188.619806,0,INDIA


In [ ]:
ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
ALL_TICKERS

['MSFT',
 'AAPL',
 'GOOG',
 'NVDA',
 'AMZN',
 'META',
 'BRK-B',
 'LLY',
 'AVGO',
 'V',
 'JPM',
 'NVO',
 'MC.PA',
 'ASML',
 'RMS.PA',
 'OR.PA',
 'SAP',
 'ACN',
 'TTE',
 'SIE.DE',
 'IDEXY',
 'CDI.PA',
 'RELIANCE.NS',
 'TCS.NS',
 'HDB',
 'BHARTIARTL.NS',
 'IBN',
 'SBIN.NS',
 'LICI.NS',
 'INFY',
 'ITC.NS',
 'HINDUNILVR.NS',
 'LT.NS']

In [ ]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed


2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed


32 LT.NS


[*********************100%%**********************]  1 of 1 completed


In [ ]:
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'


In [ ]:
stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS))

In [ ]:
# count of observations between US-EU-INDIA stocks
stocks_df.ticker_type.value_counts()

ticker_type
US       80639
EU       76986
INDIA    63265
Name: count, dtype: int64

In [ ]:
# unique tickers
stocks_df.Ticker.nunique()

33

In [ ]:
# count of observations by stock
stocks_df.Ticker.value_counts()

Ticker
LLY              13087
JPM              11121
AAPL             10933
NVO              10838
MSFT              9607
CDI.PA            8321
TTE               8185
ASML              7330
SAP               7201
HINDUNILVR.NS     7115
SBIN.NS           7113
RELIANCE.NS       7112
ITC.NS            7112
SIE.DE            7039
BRK-B             7038
AMZN              6780
NVDA              6356
INFY              6323
MC.PA             6251
RMS.PA            6251
OR.PA             6251
IBN               6058
ACN               5728
HDB               5727
LT.NS             5420
BHARTIARTL.NS     5417
TCS.NS            5388
GOOG              4955
V                 4054
AVGO              3705
IDEXY             3591
META              3003
LICI.NS            480
Name: count, dtype: int64

In [ ]:
stocks_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
AAPL,US,1980-12-12,2024-04-25
ACN,EU,2001-07-19,2024-04-25
AMZN,US,1997-05-15,2024-04-24
ASML,EU,1995-03-15,2024-04-25
AVGO,US,2009-08-06,2024-04-25
BHARTIARTL.NS,INDIA,2002-07-01,2024-04-25
BRK-B,US,1996-05-09,2024-04-25
CDI.PA,EU,1992-01-27,2024-04-25
GOOG,US,2004-08-19,2024-04-25


In [ ]:
#  average growth 365days
stocks_df[stocks_df.Year>=2020].groupby(by=['Year','ticker_type']).growth_365d.mean()

Year  ticker_type
2020  EU             1.245388
      INDIA          1.101896
      US             1.484070
2021  EU             1.582170
      INDIA          1.546424
      US             1.797571
2022  EU             1.191169
      INDIA          1.379889
      US             1.182379
2023  EU             1.246533
      INDIA          1.201110
      US             1.203253
2024  EU             1.650721
      INDIA          1.305762
      US             2.153187
Name: growth_365d, dtype: float64

In [ ]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

In [ ]:
# filtering only on stats after 2000
stocks_df[stocks_df.Date>='2000-01-01'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182423 entries, 3490 to 220889
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Open                          182423 non-null  float64       
 1   High                          182423 non-null  float64       
 2   Low                           182423 non-null  float64       
 3   Close                         182423 non-null  float64       
 4   Adj Close                     182423 non-null  float64       
 5   Volume                        182423 non-null  int64         
 6   Ticker                        182423 non-null  object        
 7   Year                          182423 non-null  int32         
 8   Month                         182423 non-null  int32         
 9   Weekday                       182423 non-null  int32         
 10  Date                          182423 non-null  datetime64[ns]
 11  growth_1d      

In [ ]:
stocks_df[stocks_df.Date=='2024-04-01'].tail()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_90d,growth_365d,growth_future_5d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_5d_future,ticker_type
194903,925.000000,984.450012,924.000000,970.599976,970.599976,4863536,LICI.NS,2024,4,0,...,1.599113,1.556710,1.026891,906.085004,953.950003,0,0.062281,1034.237352,1,INDIA
201224,17.889999,17.920000,17.500000,17.580000,17.580000,14601600,INFY,2024,4,0,...,1.003998,1.024508,1.015927,18.156000,18.777500,0,0.023891,13.940634,1,INDIA
208338,429.100006,431.350006,425.750000,426.700012,426.700012,9382308,ITC.NS,2024,4,0,...,0.987044,1.314436,1.005625,422.235004,416.537502,1,0.013124,130.246651,1,INDIA
215453,2270.000000,2295.000000,2262.000000,2285.899902,2285.899902,1427873,HINDUNILVR.NS,2024,4,0,...,0.917590,0.894051,0.992585,2266.539990,2327.240015,0,0.014436,1049.970071,0,INDIA
220873,3780.850098,3851.850098,3780.850098,3838.000000,3838.000000,1905190,LT.NS,2024,4,0,...,1.257886,2.010850,0.992144,3626.594971,3618.109985,1,0.018499,2188.619806,0,INDIA


In [ ]:
ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
ALL_TICKERS

['MSFT',
 'AAPL',
 'GOOG',
 'NVDA',
 'AMZN',
 'META',
 'BRK-B',
 'LLY',
 'AVGO',
 'V',
 'JPM',
 'NVO',
 'MC.PA',
 'ASML',
 'RMS.PA',
 'OR.PA',
 'SAP',
 'ACN',
 'TTE',
 'SIE.DE',
 'IDEXY',
 'CDI.PA',
 'RELIANCE.NS',
 'TCS.NS',
 'HDB',
 'BHARTIARTL.NS',
 'IBN',
 'SBIN.NS',
 'LICI.NS',
 'INFY',
 'ITC.NS',
 'HINDUNILVR.NS',
 'LT.NS']

In [ ]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed


2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed


32 LT.NS


[*********************100%%**********************]  1 of 1 completed


In [ ]:
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'


In [ ]:
stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS))

In [ ]:
# count of observations between US-EU-INDIA stocks
stocks_df.ticker_type.value_counts()

ticker_type
US       80639
EU       76986
INDIA    63265
Name: count, dtype: int64

In [ ]:
# unique tickers
stocks_df.Ticker.nunique()

33

In [ ]:
# count of observations by stock
stocks_df.Ticker.value_counts()

Ticker
LLY              13087
JPM              11121
AAPL             10933
NVO              10838
MSFT              9607
CDI.PA            8321
TTE               8185
ASML              7330
SAP               7201
HINDUNILVR.NS     7115
SBIN.NS           7113
RELIANCE.NS       7112
ITC.NS            7112
SIE.DE            7039
BRK-B             7038
AMZN              6780
NVDA              6356
INFY              6323
MC.PA             6251
RMS.PA            6251
OR.PA             6251
IBN               6058
ACN               5728
HDB               5727
LT.NS             5420
BHARTIARTL.NS     5417
TCS.NS            5388
GOOG              4955
V                 4054
AVGO              3705
IDEXY             3591
META              3003
LICI.NS            480
Name: count, dtype: int64

In [ ]:
stocks_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
AAPL,US,1980-12-12,2024-04-25
ACN,EU,2001-07-19,2024-04-25
AMZN,US,1997-05-15,2024-04-24
ASML,EU,1995-03-15,2024-04-25
AVGO,US,2009-08-06,2024-04-25
BHARTIARTL.NS,INDIA,2002-07-01,2024-04-25
BRK-B,US,1996-05-09,2024-04-25
CDI.PA,EU,1992-01-27,2024-04-25
GOOG,US,2004-08-19,2024-04-25


In [ ]:
#  average growth 365days
stocks_df[stocks_df.Year>=2020].groupby(by=['Year','ticker_type']).growth_365d.mean()

Year  ticker_type
2020  EU             1.245388
      INDIA          1.101896
      US             1.484070
2021  EU             1.582170
      INDIA          1.546424
      US             1.797571
2022  EU             1.191169
      INDIA          1.379889
      US             1.182379
2023  EU             1.246533
      INDIA          1.201110
      US             1.203253
2024  EU             1.650721
      INDIA          1.305762
      US             2.153187
Name: growth_365d, dtype: float64

In [ ]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

In [ ]:
# filtering only on stats after 2000
stocks_df[stocks_df.Date>='2000-01-01'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182423 entries, 3490 to 220889
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Open                          182423 non-null  float64       
 1   High                          182423 non-null  float64       
 2   Low                           182423 non-null  float64       
 3   Close                         182423 non-null  float64       
 4   Adj Close                     182423 non-null  float64       
 5   Volume                        182423 non-null  int64         
 6   Ticker                        182423 non-null  object        
 7   Year                          182423 non-null  int32         
 8   Month                         182423 non-null  int32         
 9   Weekday                       182423 non-null  int32         
 10  Date                          182423 non-null  datetime64[ns]
 11  growth_1d      

In [ ]:
stocks_df[stocks_df.Date=='2024-04-01'].tail()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_90d,growth_365d,growth_future_5d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_5d_future,ticker_type
194903,925.000000,984.450012,924.000000,970.599976,970.599976,4863536,LICI.NS,2024,4,0,...,1.599113,1.556710,1.026891,906.085004,953.950003,0,0.062281,1034.237352,1,INDIA
201224,17.889999,17.920000,17.500000,17.580000,17.580000,14601600,INFY,2024,4,0,...,1.003998,1.024508,1.015927,18.156000,18.777500,0,0.023891,13.940634,1,INDIA
208338,429.100006,431.350006,425.750000,426.700012,426.700012,9382308,ITC.NS,2024,4,0,...,0.987044,1.314436,1.005625,422.235004,416.537502,1,0.013124,130.246651,1,INDIA
215453,2270.000000,2295.000000,2262.000000,2285.899902,2285.899902,1427873,HINDUNILVR.NS,2024,4,0,...,0.917590,0.894051,0.992585,2266.539990,2327.240015,0,0.014436,1049.970071,0,INDIA
220873,3780.850098,3851.850098,3780.850098,3838.000000,3838.000000,1905190,LT.NS,2024,4,0,...,1.257886,2.010850,0.992144,3626.594971,3618.109985,1,0.018499,2188.619806,0,INDIA


In [57]:
stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS))

In [58]:
# count of observations between US-EU-INDIA stocks
stocks_df.ticker_type.value_counts()

ticker_type
US       80717
EU       77058
INDIA    63334
Name: count, dtype: int64

In [31]:
# unique tickers
stocks_df.Ticker.nunique()

33

In [32]:
# count of observations by stock
stocks_df.Ticker.value_counts()

Ticker
LLY              13094
JPM              11128
AAPL             10940
NVO              10845
MSFT              9614
CDI.PA            8327
TTE               8192
ASML              7337
SAP               7208
HINDUNILVR.NS     7121
SBIN.NS           7119
RELIANCE.NS       7118
ITC.NS            7118
BRK-B             7045
SIE.DE            7045
AMZN              6788
NVDA              6363
INFY              6330
MC.PA             6257
RMS.PA            6257
OR.PA             6257
IBN               6065
ACN               5735
HDB               5734
LT.NS             5426
BHARTIARTL.NS     5423
TCS.NS            5394
GOOG              4962
V                 4061
AVGO              3712
IDEXY             3598
META              3010
LICI.NS            486
Name: count, dtype: int64

In [33]:
stocks_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
AAPL,US,1980-12-12,2024-05-06
ACN,EU,2001-07-19,2024-05-06
AMZN,US,1997-05-15,2024-05-06
ASML,EU,1995-03-15,2024-05-06
AVGO,US,2009-08-06,2024-05-06
BHARTIARTL.NS,INDIA,2002-07-01,2024-05-06
BRK-B,US,1996-05-09,2024-05-06
CDI.PA,EU,1992-01-27,2024-05-06
GOOG,US,2004-08-19,2024-05-06


In [34]:
#  average growth 365days
stocks_df[stocks_df.Year>=2020].groupby(by=['Year','ticker_type']).growth_365d.mean()

Year  ticker_type
2020  EU             1.245388
      INDIA          1.101896
      US             1.484070
2021  EU             1.582170
      INDIA          1.546424
      US             1.797571
2022  EU             1.191169
      INDIA          1.379889
      US             1.182379
2023  EU             1.246533
      INDIA          1.201110
      US             1.203253
2024  EU             1.640018
      INDIA          1.302174
      US             2.160563
Name: growth_365d, dtype: float64

In [35]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

In [36]:
# filtering only on stats after 2000
stocks_df[stocks_df.Date>='2000-01-01'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182642 entries, 3490 to 221108
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Open                          182642 non-null  float64       
 1   High                          182642 non-null  float64       
 2   Low                           182642 non-null  float64       
 3   Close                         182642 non-null  float64       
 4   Adj Close                     182642 non-null  float64       
 5   Volume                        182642 non-null  int64         
 6   Ticker                        182642 non-null  object        
 7   Year                          182642 non-null  int32         
 8   Month                         182642 non-null  int32         
 9   Weekday                       182642 non-null  int32         
 10  Date                          182642 non-null  datetime64[ns]
 11  growth_1d      

In [37]:
stocks_df[stocks_df.Date=='2024-04-01'].tail()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_90d,growth_365d,growth_future_5d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_5d_future,ticker_type
195091,925.000000,984.450012,924.000000,970.599976,970.599976,4863536,LICI.NS,2024,4,0,...,1.599113,1.556710,1.026891,906.085004,953.950003,0,0.062281,1034.237352,1,INDIA
201418,17.889999,17.920000,17.500000,17.580000,17.580000,14601600,INFY,2024,4,0,...,1.003998,1.024508,1.015927,18.156000,18.777500,0,0.023891,13.940634,1,INDIA
208539,429.100006,431.350006,425.750000,426.700012,426.700012,9382308,ITC.NS,2024,4,0,...,0.987044,1.314436,1.005625,422.235004,416.537502,1,0.013124,130.246651,1,INDIA
215660,2270.000000,2295.000000,2262.000000,2285.899902,2285.899902,1427873,HINDUNILVR.NS,2024,4,0,...,0.917590,0.894050,0.992585,2266.539990,2327.240015,0,0.014436,1049.970071,0,INDIA
221086,3780.850098,3851.850098,3780.850098,3838.000000,3838.000000,1905190,LT.NS,2024,4,0,...,1.257886,2.010850,0.992144,3626.594971,3618.109985,1,0.018499,2188.619806,0,INDIA


### Question 3: Answer

In [39]:
# Q3 Answer

---
### Question 4: Trying Another Technical Indicators strategy

**What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?**


First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31).
If you encounter any difficulties running the Colab - you can download it using this [link](https://drive.google.com/file/d/1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb/view?usp=sharing).

Let's assume you've learned about the awesome **CCI indicator** ([Commodity Channel Index](https://www.investopedia.com/terms/c/commoditychannelindex.asp)), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (`Date.dt.dayofweek()==4`).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years?
One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit.
You need to sum these results over all trades (460 times in 10 years).

Additional:
  * Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
  * are you still profitable on those trades?


### Question 4: Answer

In [41]:
# Q4 Answer


---
### [EXPLORATORY] Question 5: Finding Your Strategy for IPOs

You've seen in the first questions that the median and average investments are negative in IPOs, and you can't blindly invest in all deals.

How would you correct/refine the approach? Briefly describe the steps and the data you'll try to get (it should be generally feasible to do it from public sources - no access to internal data of companies)?

E.g. (some ideas) Do you want to focus on the specific vertical? Do you want to build a smart comparison vs. existing stocks on the market? Or you just will want to get some features (which features?) like total number of people in a company to find a segment of "successful" IPOs?

### Question 5: Answer

In [40]:
# Q5 Answer



---
## Submitting the solutions

Form for submitting: https://courses.datatalks.club/sma-zoomcamp-2024/homework/hw02

---
